# Randomly Generated a World Map

### Building a random world map each time that a new character is created.
I wanted my Flask Game to be focused on randomly generated content. So when a new user creates an account, a world is also created for them. This way I can design a game like a statistician, not like a storyteller. I also wanted to build from a data science perspecitve, using tools like D3.js and Python in order to develop my skills in these applications. 

[**You can play the game here**](http://williamjeffreyharding.com/game/welcome). I haven't added any bad guys or quests but you can explore the terrain. Hoping to add more content to this game this year.

### Defining the traits of the world
In my game, I set up the definitions of objects, and their traits. D3.js will later define the business rules and how objects interact with each other. 

[here's an example of how objects interact](https://github.com/BillmanH/Flask-Game-Front-End/blob/master/activity_map_boarder.html)

In [15]:
#This code is just to set up the data that I'll use in the rest of this module. 
import pandas as pd
import numpy as np
import boto
import yaml

#terrain data 
ddf = pd.read_csv("/home/billmanh/Downloads/Play/Flask RPG/Flask RPG - Terrain Detail.tsv",sep="\t")
#bestiary - beasts are not implemented yet but I'll be working on them shortly.
bdf = pd.read_csv("/home/billmanh/Downloads/Play/Flask RPG/Flask RPG - Bestiary.tsv",sep="\t")

terrain = {}
for row in ddf.index:
    terrain[ddf.ix[row,'Code']]=yaml.load(ddf.ix[row].to_json())

The only thing that has human input at this point is the description of the attributes in the world. Each region on the map contains a type, which has various danger levels, descriptions, textures and creatures. 

In [9]:
ddf.head()

,Code,Type Name,Type,Danger Level,Creature Probability,Treasure Level,Description,Detailed Description,creatures,Terrain Textures
0,fo,forest,normal,4,0.5,3,a quiet deciduous forest,NaN,goblin,"[ {'name':'Tree', 'density':6,..."
1,sw,swamp,normal,8,0.8,2,a desolate swamp,NaN,goblin,"[ {'name':'Fern', 'abundance':..."
2,mt,mountain,normal,6,0.4,5,a rugged mountain range,NaN,goblin,"[ {'name':'Boulder', 'density'..."
3,sa,savanna,normal,3,0.1,3,a dry savana,NaN,"goblin,tiger","[ {'name':'Umbrella Thorn', 'd..."
4,pl,plain,normal,2,0.1,2,a grassy plain,NaN,tiger,"[ {'name':'Grass', 'abundance'..."


Each Texture type has it's own business logic, which is deployed in the [cleint side code](https://github.com/BillmanH/Flask-Game-Front-End/tree/master/terrain) when the specified texture is called and for the number of times in the texture density. I can add or remove elements from the world by editing a google sheet and then downloading as csv. 

The bestiary isn't working yet, but I'm using this as a template to get the textures going. 

In [10]:
bdf.head()

,index,name,move,size,move type,render type,group min,group max,color
0,goblin,Goblin,5,5,swarm,pack,5,10,#3E8E3E
1,tiger,Tiger,10,6,swarm,alone,1,1,#e08d3c
2,scorpion,Giant Scorpion,3,8,swarm,scatter,1,2,#716D6F
3,yeti,Yeti,5,15,swarm,alone,1,1,#a0d6b4
4,raider,Raider,6,10,swarm,pack,2,3,#4a3b29


The details of the terrain are pulled up for the local area when it is rendered to the client, so I only need to know the types of terrain to be placed. Note that the terrain textures are included in the terrain details. The code is important as it will map the terrain to the specific details of that terrain.

In [11]:
terrain['ds']

{'Code': 'ds',
 'Creature Probability': 0.1,
 'Danger Level': 1,
 'Description': 'a barren desert',
 'Detailed Description': None,
 'Terrain Textures': "[         {'name':'Sand',         'abundance':300,         'density':5} ]",
 'Treasure Level': 1,
 'Type': 'normal',
 'Type Name': 'desert',
 'creatures': 'scorpion'}

In [12]:
for t in list(terrain.keys()):
    print(t, " : ",terrain[t]['Type Name']," : ",terrain[t]['Description'])

ds  :  desert  :  a barren desert
pl  :  plain  :  a grassy plain
fo  :  forest  :  a quiet deciduous forest
oa  :  oasis  :  an oasis
ic  :  tundra  :  a tundra
sw  :  swamp  :  a desolate swamp
sa  :  savanna  :  a dry savana
mt  :  mountain  :  a rugged mountain range


I'm keeping a space for "special" types of terrain. This way I can add quest destinations later on.

In [16]:
terrain_types = [t for t in list(terrain.keys()) if terrain[t]['Type']=='normal']
terrain_types

['ds', 'pl', 'fo', 'ic', 'sw', 'sa', 'mt']

## Building a world: create a square and fill it up. 
The game starts with a lot of blank spaces, when you reach one side of the earth you are automatically placed on the other side so this world appears spherical as you travel from area to area. 

[you can see how the code is executed on the server here](https://github.com/BillmanH/Flask-Game-Front-End/blob/master/backend/MapData_RandomlyGenerateMap.py)

In [47]:
grid = [10,10]
df = pd.DataFrame(columns=range(grid[1]),index=range(grid[0]))
dfMeta = {}
df

,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filling that grid with values
Then you go square by square and place random terrain until it is filled up. This is currently done synchronously so 10X10 is about as large as I want for a starting world. Later on I'll add an asynchronous request to build a much larger world. I'll probably use that as my free-paid tier barrier. 

In [48]:
def get_random_chord(df):
    x = np.random.choice(df[df.isnull().any(axis=0)].index.tolist(),1)[0]
    y = np.random.choice(df[df.isnull().any(axis=1)].index.tolist(),1)[0]
    coord = [x,y]
    return coord

def count_remaining_na():
    num = sum([df[[n]].isnull().sum().tolist()[0] for n in df.columns])
    return num

def listBlankSpaces():
    places = df.T.to_dict()
    blankPlaces = []
    for x in places.keys():
        for y in places[x].keys():
            if pd.isnull(places[x][y]):
                #print(places[x][y],"is not a string")
                blankPlaces.append([x,y])
    return blankPlaces

def place_terrain(coord):
    t_type = np.random.choice(terrain_types,1)[0]
    df.loc[coord[0],coord[1]] = t_type
    return df

def getMostCommonTerrain(coord):
    neighbors = []
    #one for each neighboring spot
    try:
        neighbors.append(df.loc[coord[0]+1,coord[1]+1])
    except:
        pass
    try:    
        neighbors.append(df.loc[coord[0],coord[1]+1])
    except:
        pass
    try:
        neighbors.append(df.loc[coord[0]-1,coord[1]+1])
    except:
        pass
    try:
        neighbors.append(df.loc[coord[0]+1,coord[1]])
    except:
        pass
    try:
        neighbors.append(df.loc[coord[0]-1,coord[1]])
    except:
        pass
    try:
        neighbors.append(df.loc[coord[0]+1,coord[1]-1])
    except:
        pass
    try:
        neighbors.append(df.loc[coord[0],coord[1]-1])
    except:
        pass
    try:
        neighbors.append(df.loc[coord[0]-1,coord[1]-1])
    except:
        pass
    #find the terrainthat is most abundant
    try:
        neighbors = [neigh for neigh in neighbors if neigh in terrain_types]  #removing nan and things that aren't terrain types
        mostCommon = neighbors[np.argmax(np.unique(neighbors,return_counts=True))]
        if np.argmax(np.unique(neighbors,return_counts=True))>1:
            t = mostCommon
        else:
            t = np.random.choice(terrain_types,1)[0]
    except:
        t = np.random.choice(terrain_types,1)[0]
    return t

def place_best_terrain(coord):
    t = getMostCommonTerrain(coord)
    if t in list(dfMeta.keys()):
        dfMeta[t] = dfMeta[t] + 1
    else:
        dfMeta[t] = 1
    df.loc[coord[0],coord[1]] = t
    return df

def place_specific_terrain(t,coord):
    df.loc[coord[0],coord[1]] = t
    return df

def get_current_terrain(coord):
    ter = df.ix[coord[0],coord[1]]
    return ter


def find_blank_neighbor(coord):
    taken = [[-1,-1]]
    while len(taken)<8:
        if check_if_coord_is_na(coord):
            return coord
        else:
            coord,taken = find_neighboring_spot(coord,taken)
    return [[-1,-1]]
    pass

Terrain detail is loaded independently so there is no need for that level of detail here. We only need to place the location code and terrain for that area. 
* Create terrain in steps:
    * Choose random spot from a list of remaining blank spots
    * Get a list of all of the terrain in neighboring areas
        * Place terrain that occurs most commonly in that area
        * If most of the neighboring terrain is blank, choose a terrain randomly
        * Place the chosen terrain in that spot

In [49]:
while count_remaining_na() > 0:
    #print(count_remaining_na(),coord)
    blanks = listBlankSpaces()
    coord = blanks[np.random.choice(len(blanks))]
    place_best_terrain(coord)

In [50]:
df

,0,1,2,3,4,5,6,7,8,9
0,mt,pl,pl,ds,ds,mt,mt,sa,ic,pl
1,pl,pl,pl,sa,ds,ds,mt,ic,fo,ic
2,sw,pl,pl,pl,pl,fo,ic,fo,ds,mt
3,mt,ds,pl,sw,ds,ic,ic,ds,ds,ds
4,sw,sw,sw,ds,fo,mt,ic,pl,ic,fo
5,ds,pl,sw,sw,mt,mt,pl,pl,pl,fo
6,ds,fo,fo,sw,fo,mt,pl,ic,pl,sw
7,fo,fo,fo,mt,mt,sw,sw,sw,sw,pl
8,fo,ic,fo,sa,pl,sw,sw,sw,fo,sa
9,fo,ic,sa,sa,ic,mt,mt,sw,sw,ic


## Data about the world: 
I'm creating a separate dataset of metadata for that world. Eventually I might build out something that allows users the ability to tweak the parameters of the world and thus change the experience. 

In [51]:
dfMeta

{'ds': 13, 'fo': 16, 'ic': 13, 'mt': 14, 'pl': 20, 'sa': 6, 'sw': 18}

In [52]:
meta = pd.DataFrame(dfMeta,index=["quantity"]).T
meta["percent of world"] = meta["quantity"]/meta.quantity.sum()
meta['id'] = meta.index
meta.index = [terrain[x]['Type Name'] for x in meta.index]

In [53]:
meta

,quantity,percent of world,id
desert,13,0.13,ds
forest,16,0.16,fo
tundra,13,0.13,ic
mountain,14,0.14,mt
plain,20,0.20,pl
savanna,6,0.06,sa
swamp,18,0.18,sw


## Once the map is drawn it must be converted back to JSON
Terrain attriutes must be added to each item. 

Each account gets two distinct files, one character and one map. They map together with a common key so that each player is only playing on thier map. 


In [54]:
UserID = "william.jeffrey.harding@gmail.com"  #must map back to a character, otherwise you will never see it. 
worldMap = {}

In [55]:
for x in range(grid[0]):
    for y in range(grid[1]):
        worldMap[str(x)+":"+str(y)] = {"Terrain Code":df.loc[y,x],
                                      "y":y,"x":x}

In order to keep the frame as light as possible, I'm only exporting the coordinates and the terrain code. Terrain details are loaded at runtime, only for elements that are being deployed.[you can see this in action here](https://github.com/BillmanH/Flask-Game-Front-End/blob/master/backend/BOTO_functions.py)

In [46]:
print(worldMap['0:1'],
     worldMap['0:2'],
     worldMap['0:3'],
     worldMap['0:4'])

{'Terrain Code': 'pl', 'y': 1, 'x': 0} {'Terrain Code': 'pl', 'y': 2, 'x': 0} {'Terrain Code': 'mt', 'y': 3, 'x': 0} {'Terrain Code': 'ds', 'y': 4, 'x': 0}


## Adding creatures
This feature really isn't ready. I'm just starting to POC this feature in Jupyter notebooks now. However, you can see that there is a consistent organization of json objects where:
* Each region has terrain
* Each terrain has both terrain textures and monsters
* The properties of the feature is in the json, the business logic is in the Javascript

TODO:
* add google doc sheet for monsters
* add insert monsters into terrain

currently working with only one kind of monster:

In [58]:
bdf.head()

,index,name,move,size,move type,render type,group min,group max,color
0,goblin,Goblin,5,5,swarm,pack,5,10,#3E8E3E
1,tiger,Tiger,10,6,swarm,alone,1,1,#e08d3c
2,scorpion,Giant Scorpion,3,8,swarm,scatter,1,2,#716D6F
3,yeti,Yeti,5,15,swarm,alone,1,1,#a0d6b4
4,raider,Raider,6,10,swarm,pack,2,3,#4a3b29


In [59]:
def addCreaturesToMap(worldMap,terrain):
    for k in worldMap.keys():
        tk = worldMap[k]['Terrain Code']

In [60]:
k = list(worldMap.keys())[1]
tk = worldMap[k]['Terrain Code']
k,terrain[tk]

('2:2',
 {'Code': 'pl',
  'Creature Probability': 0.1,
  'Danger Level': 2,
  'Description': 'a grassy plain',
  'Detailed Description': None,
  'Terrain Textures': "[         {'name':'Grass',         'abundance':150,         'density':5},          {'name':'Rock',         'abundance':5,         'density':5} ]",
  'Treasure Level': 2,
  'Type': 'normal',
  'Type Name': 'plain',
  'creatures': 'tiger'})

In [61]:
terrain[tk]['Creature Probability']

0.1

NameError: name 'k' is not defined